In [31]:
# Imports
import torch
import torchvision # torch package for vision related things
import torch.nn.functional as F  # Parameterless functions, like (some) activation functions
import torchvision.datasets as datasets  # Standard datasets
import torchvision.transforms as transforms  # Transformations we can perform on our dataset for augmentation
from torch import optim  # For optimizers like SGD, Adam, etc.
from torch import nn  # All neural network modules
from torch.utils.data import DataLoader  # Gives easier dataset managment by creating mini batches etc.
from tqdm import tqdm  # For nice progress bar!
from sklearn.model_selection import KFold

import numpy as np
import os
import pandas as pd

import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data import random_split
import torch.nn as nn

In [32]:
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler
from functools import partial

In [33]:
"""
(Hyper)parameters of our convolutional neural network found by tuning with Ray Tune (see further on)

"""
input_size = 3 * 100
num_classes = 3
learning_rate = 0.045421358413054946
batch_size = 10
num_epochs = 5

## Multi Layer Perceptron
After extracting relevant features from the PAC matrices, using a Convolutional Neural Network, these features are combined with the IL- and BLA-bandpower and fed to a fully connected neural network or multi-layer perceptron.

The Multi-Layer Perceptron consists of 3 fully connected layers, each followed by a rectified linear activation function and a final fully connected layer (followed by a softmax function???)


In [34]:
"""
Class inherits from nn.Module, a pytorch neural network of choice, must inherit from base class nn.Module.
"""
class MLP(nn.Module):
    def __init__(self, input_size = input_size, num_classes = num_classes):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, 25)
        self.fc3 = nn.Linear(25, 10)
        self.fc4 = nn.Linear(10, num_classes)

    def forward(self, x):
        """
        Forward propagation function.
        
        self: self
        x: tensor of shape (1, input_size)
        
        returns: tensor of shape (1, num_classes) serving as a feature vector.
        """
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

In [35]:
"""
Set device cuda for GPU if it's available otherwise run on the CPU
"""
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Load labels

In [36]:
"""
Load labels from csv file
"""
labels = np.genfromtxt('../Data/Labels/avg_smooth.csv', delimiter=',')

## Quantify raw label values
After reading the raw label values, they're quantified to 3 discrete levels: 0, 1 or 2.

0: 'No fear apparent in rodents brain activity'

1: 'A small amount of fear could be noticed, but too noisey to be sure.'

2: 'Rodent is fearful'

In [37]:
"""
Make labels categorical: 3 quantification levels
"""
df_labels = pd.DataFrame(labels)
df_labels = pd.cut(df_labels[0],bins=[0, 0.013, 0.017, 0.03],labels=[0,1,2])
labels = df_labels.to_numpy()

print('\nAmount of non-fearful rodents:' + str(len([i for i in list(labels) if i==0])))
print('Amount of little fearful rodents:' + str(len([i for i in list(labels) if i==1])))
print('Amount of fearful rodents:' + str(len([i for i in list(labels) if i==2])))


Amount of non-fearful rodents:200
Amount of little fearful rodents:204
Amount of fearful rodents:196


## Baseline model
After loading in all the correct labels, a very basic baseline model is built.

In this model, each test sample will get labeled as the most occuring label in the train set.
A complex model, being built later on, should be expected to have an accuracy at least better than this baseline.

In [38]:
"""
Split data into 70% train data, 30 % test data
"""
train_labels = labels[:int(len(labels)*7/10)]
test_labels = labels[int(len(labels)*7/10):]

print('Most occuring label in testset: ' + str(np.bincount(labels).argmax()))

count_arr = np.bincount(test_labels)

accuracy = count_arr[1]/len(test_labels)
print('Baseline accuracy: ' + str(accuracy))

Most occuring label in testset: 1
Baseline accuracy: 0.24444444444444444


## Load all feature vectors from .npy-files

In [39]:
"""
Load .npy files into one big array
"""
data = []
dir = '../Data/input_MLP/'
pac = np.load('../Data/PAC_afterCNN.npy')
for filename in os.listdir(dir):
    sample = np.load(dir+filename)
    sample = np.append(sample, pac)
    data.append(sample.flatten())
data = np.array(data)
print('Feature shapes: ' + str(data.shape), '\nLabel vector shape: ' + str(labels.shape))

Feature shapes: (600, 300) 
Label vector shape: (600,)


In [40]:
"""
Transform data to torch tensors
"""
tensor_x = torch.Tensor(data)
tensor_y = torch.Tensor(labels)
tensor_y = tensor_y.type(torch.LongTensor)

In [41]:
"""
Create dataset and data loader
"""
dataset = TensorDataset(tensor_x,tensor_y)

"""
Split into train and test sets
"""
test_size = int(0.3*len(dataset))
train_size = len(dataset) - test_size

train_data,test_data = random_split(dataset,[train_size,test_size])
print(f"Length of Train Data : {len(train_data)}")
print(f"Length of Test Data : {len(test_data)}")

train_loader = DataLoader(train_data, batch_size, shuffle = False, num_workers = 4, pin_memory = True)
test_loader = DataLoader(test_data, batch_size, shuffle = False, num_workers = 4, pin_memory = True)

Length of Train Data : 420
Length of Test Data : 180


In [42]:
"""
Initialize network
""" 
model = MLP(input_size, num_classes).to(device)
"""
Loss and optimizer
""" 
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [43]:
def reset_weights(m):
    """
    Try resetting model weights to avoid
    weight leakage.

    m: a PyTorch model

    return: None
    """
    for layer in m.children():
        if hasattr(layer, 'reset_parameters'):
            #print(f'Reset trainable parameters of layer = {layer}')
            layer.reset_parameters()

## Define accuracy-function

In [44]:
"""
 Check accuracy on training & test to see how good the model is
 
 loader: a pytorch dataloader
 model: a pytorch model
 
 returns: model accuracy as a numerical value
"""
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            #x = x.reshape(x.shape[0], -1)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

    model.train()
    return num_correct/num_samples

In [45]:
"""
 Check accuracy on training & test to see how good the model performs within each class
 
 loader: a pytorch dataloader
 model: a pytorch model
 
 returns: a list of model accuracies as a numerical value
"""
def check_accuracy_per_class(loader, model):
    num_correct = [0.0, 0.0, 0.0]
    num_samples = [0.0, 0.0, 0.0]
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            #x = x.reshape(x.shape[0], -1)

            scores = model(x)
            _, predictions = scores.max(1)
            for (prediction, y_i) in zip(predictions, y):
                prediction = prediction.item()
                y_i = y_i.item()
                if(y_i == 0):
                    if(prediction == y_i): 
                        num_correct[0] += 1
                    num_samples[0] += 1
                elif(y_i == 1):
                    if(prediction == y_i): 
                        num_correct[1] += 1
                    num_samples[1] += 1
                elif(y_i == 2):
                    if(prediction == y_i): 
                        num_correct[2] += 1
                    num_samples[2] += 1
    model.train()
    accuracy = [0.0, 0.0, 0.0]
    if num_samples[0] != 0:
        accuracy[0] = num_correct[0]/num_samples[0]
    if num_samples[1] != 0:
        accuracy[1] = num_correct[1]/num_samples[1]
    if num_samples[2] != 0:
        accuracy[2] = num_correct[2]/num_samples[2]
    return accuracy

## Apply crossvalidation
Apply 5-fold crossvalidation with CrossEntropy loss and Adam optimizer.

In [46]:
"""
Define the 5-fold Cross Validator
"""
k_folds = 5
kfold = KFold(n_splits=k_folds, shuffle=False)

In [48]:
train_acc = []
test_acc = []

train_acc_per_class = []
test_acc_per_class = []

for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset)):
    print(f'Fold {fold}')
    model.apply(reset_weights)
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
    trainloader = torch.utils.data.DataLoader(
                      dataset, batch_size=10, sampler=train_subsampler)
    testloader = torch.utils.data.DataLoader(
                      dataset, batch_size=10, sampler=test_subsampler)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    for epoch in range(num_epochs):
        for batch_idx, (input_, labels) in enumerate(tqdm(trainloader)):
            # Get data to cuda if possible
            input_ = input_.to(device=device)
            targets = labels.to(device=device)

            # forward
            scores = model(input_)
            loss = criterion(scores, targets)

            # backward
            optimizer.zero_grad()
            loss.backward()

            # gradient descent or adam step
            optimizer.step()
        train_acc.append(check_accuracy(trainloader, model).item()*100)
        test_acc.append(check_accuracy(testloader, model).item()*100)
        train_acc_per_class.append(check_accuracy_per_class(trainloader, model))
        test_acc_per_class.append(check_accuracy_per_class(testloader, model))
    print('Train Accuracy for fold %d: %d %%' % (fold, 100.0 * check_accuracy(trainloader, model)))
    print('Test Accuracy for fold %d: %d %%' % (fold, 100.0 * check_accuracy(testloader, model)))
    print('Train Accuracies per class [Label 0, Label 1, Label 2] for fold %d: ' % (fold))
    print(*check_accuracy_per_class(trainloader, model))
    print('Test Accuracies per class [Label 0, Label 1, Label 2] for fold %d: ' % (fold))
    print(*check_accuracy_per_class(testloader, model))
print('Averaged Train Accuracy over %d k-folds: %d %%' % (k_folds, np.array(train_acc).mean()))
print('Averaged Test Accuracy over %d k-folds: %d %%' % (k_folds, np.array(test_acc).mean()))
print('Averaged Train Accuracy per class', *np.matrix(train_acc_per_class).mean(0))
print('Averaged Test Accuracy per class', *np.matrix(test_acc_per_class).mean(0))

Fold 0


100%|█████████████████████████████████████████████████████████████████████████████████| 48/48 [00:00<00:00, 510.67it/s]


Train Accuracy for fold 0: 36 %
Test Accuracy for fold 0: 25 %
Train Accuracies per class [Label 0, Label 1, Label 2] for fold 0: 
0.0 1.0 0.0
Test Accuracies per class [Label 0, Label 1, Label 2] for fold 0: 
0.0 1.0 0.0
Fold 1


100%|█████████████████████████████████████████████████████████████████████████████████| 48/48 [00:00<00:00, 363.63it/s]


Train Accuracy for fold 1: 36 %
Test Accuracy for fold 1: 19 %
Train Accuracies per class [Label 0, Label 1, Label 2] for fold 1: 
0.0 0.0 1.0
Test Accuracies per class [Label 0, Label 1, Label 2] for fold 1: 
0.0 0.0 1.0
Fold 2


...

Train Accuracy for fold 2: 39 %
Test Accuracy for fold 2: 5 %
Train Accuracies per class [Label 0, Label 1, Label 2] for fold 2: 
0.0 0.0 1.0
Test Accuracies per class [Label 0, Label 1, Label 2] for fold 2: 
0.0 0.0 1.0
Fold 3


100%|█████████████████████████████████████████████████████████████████████████████████| 48/48 [00:00<00:00, 539.32it/s]


Train Accuracy for fold 3: 39 %
Test Accuracy for fold 3: 9 %
Train Accuracies per class [Label 0, Label 1, Label 2] for fold 3: 
1.0 0.0 0.0
Test Accuracies per class [Label 0, Label 1, Label 2] for fold 3: 
1.0 0.0 0.0
Fold 4


100%|█████████████████████████████████████████████████████████████████████████████████| 48/48 [00:00<00:00, 494.83it/s]


Train Accuracy for fold 4: 34 %
Test Accuracy for fold 4: 30 %
Train Accuracies per class [Label 0, Label 1, Label 2] for fold 4: 
0.0 1.0 0.0
Test Accuracies per class [Label 0, Label 1, Label 2] for fold 4: 
0.0 1.0 0.0
Averaged Train Accuracy over 5 k-folds: 36 %
Averaged Test Accuracy over 5 k-folds: 18 %
Averaged Train Accuracy per class [[0.20024691 0.32       0.47953757]]
Averaged Test Accuracy per class [[0.20105263 0.32       0.48      ]]


## Hyperparameter Tuning with Ray Tune
As noted before some parameters where acquired in their optimal form by performing hyperparameter tuning.  
The library used for this was Ray Tune.  

Following hyperparameters where tuned:
- number of epochs
- learning rate
- batch size

with following optimal values:  
- number of epochs = 5
- learning_rate = 0.045421358413054946
- batch_size = 10

In [17]:
config = {
    "num_epochs": tune.choice([2, 5, 10]),
    "lr": tune.loguniform(1e-4, 1e-1),
    "batch_size": tune.choice([5, 10, 50, 100])
}
max_num_epochs = 11

In [18]:
"""
Ray Tune requires one method to contain all necessary network definitions and methods.

config: dictionary with configuration hyperparameters
checkpoint_dir: not used, but could be used to load earlier configuration

returns: None
"""
def train(config, checkpoint_dir=None):
    model = MLP().to(device)
    model.apply(reset_weights)
    # Loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=config["learning_rate"])
    
    if checkpoint_dir:
        model_state, optimizer_state = torch.load(
            os.path.join(checkpoint_dir, "checkpoint"))
        net.load_state_dict(model_state)
        optimizer.load_state_dict(optimizer_state)
    batch_size = config["batch_size"]
    train_loader = DataLoader(train_data, batch_size, shuffle = False, num_workers = 2, pin_memory = True)
    test_loader = DataLoader(test_data, batch_size, shuffle = False, num_workers = 2, pin_memory = True)
    for epoch in range(config["num_epochs"]):
        running_loss = 0
        epoch_steps = 0
        for batch_idx, (data, labels) in enumerate(tqdm(train_loader)):
            current_loss = 0
            # Get data to cuda if possible
            data = data.to(device=device)
            targets = labels.to(device=device)

            # forward
            scores = model(data)
            loss = criterion(scores, targets)

            # backward
            optimizer.zero_grad()
            loss.backward()

            # gradient descent or adam step
            optimizer.step()

            #print statistics
            running_loss += loss.item()
            epoch_steps += 1
            if batch_idx%500 == 499:
                print("[%d, %5d] loss: %.3f" % (epoch + 1, i + 1,
                                                running_loss / epoch_steps))
                running_loss = 0.0
                
                
        test_loss = 0
        test_steps = 0
        total = 0
        correct = 0
        for batch_idx, (data, labels) in enumerate(tqdm(test_loader)):
            with torch.no_grad():
                data = data.to(device=device)
                targets = labels.to(device=device)
                scores = model(data)
                _, predictions = scores.max(1)
                total += predictions.size(0)
                correct += (predictions == targets).sum().item()
                loss = criterion(scores, targets)
                test_loss += loss.cpu().numpy()
                test_steps+=1
        with tune.checkpoint_dir(epoch) as checkpoint_dir:
            path = os.path.join(checkpoint_dir, "checkpoint")
            torch.save((model.state_dict(), optimizer.state_dict()), path)
        tune.report(loss=(test_loss/test_steps), accuracy=correct/total)
    print("Finished")

In [19]:
"""
num_samples: amount of hyperparameter value combinations to check
gpus_per_trial: amount of gpus available

return: None
"""
def main(num_samples = 10, gpus_per_trial=1):
    config = {
    "num_epochs": tune.choice([4, 5, 10]),
    "learning_rate": tune.loguniform(1e-4, 1e-1),
    "batch_size": tune.choice([10, 30, 50, 100])
    }
    scheduler = ASHAScheduler(
        metric="loss",
        mode="min",
        max_t = max_num_epochs,
        grace_period=1,
        reduction_factor=2)
    reporter = CLIReporter(metric_columns=["loss", "accuray", "train_iteration"])
    result = tune.run(
            partial(train),
            resources_per_trial={"cpu": 1, "gpu": gpus_per_trial},
            config=config,
            num_samples = num_samples,
            scheduler=scheduler,
            progress_reporter=reporter)
    best_trial = result.get_best_trial("loss", "min", "last")
    print("Best trial config: {}".format(best_trial.config))
    print("Best trial final validation loss: {}".format(
        best_trial.last_result["loss"]))
    print("Best trial final validation accuracy: {}".format(
        best_trial.last_result["accuracy"]))
    #best_trained_model = model(best_trial)

In [20]:
main(num_samples=10, gpus_per_trial=1)

2021-12-22 15:38:20,134	INFO services.py:1338 -- View the Ray dashboard at http://127.0.0.1:8266
2021-12-22 15:38:31,449	WARNING experiment.py:255 -- No name detected on trainable. Using DEFAULT.
2021-12-22 15:38:31,454	INFO registry.py:69 -- Detected unknown callable for trainable. Converting to class.


== Status ==
Current time: 2021-12-22 15:38:32 (running for 00:00:00.78)
Memory usage on this node: 16.5/47.6 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Resources requested: 0/24 CPUs, 0/2 GPUs, 0.0/37.98 GiB heap, 0.0/18.99 GiB objects
Result logdir: C:\Users\Tomas\ray_results\DEFAULT_2021-12-22_15-38-31
Number of trials: 10/10 (10 PENDING)
+---------------------+----------+-------+--------------+-----------------+--------------+
| Trial name          | status   | loc   |   batch_size |   learning_rate |   num_epochs |
|---------------------+----------+-------+--------------+-----------------+--------------|
| DEFAULT_d5cba_00000 | PENDING  |       |           50 |     0.000109278 |           10 |
| DEFAULT_d5cba_00001 | PENDING  |       |           50 |     0.0422751   |            5 |
| DEFAULT_d5cba_00002 | PENDING  |       |           30 |     0.0195957   |           10 |
| DEFAULT_d5cba_00003 | PENDIN

2021-12-22 15:38:38,780	ERROR syncer.py:111 -- Log sync requires rsync to be installed.


== Status ==
Current time: 2021-12-22 15:38:43 (running for 00:00:11.77)
Memory usage on this node: 17.2/47.6 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Resources requested: 1.0/24 CPUs, 1.0/2 GPUs, 0.0/37.98 GiB heap, 0.0/18.99 GiB objects
Result logdir: C:\Users\Tomas\ray_results\DEFAULT_2021-12-22_15-38-31
Number of trials: 10/10 (9 PENDING, 1 RUNNING)
+---------------------+----------+-----------------+--------------+-----------------+--------------+
| Trial name          | status   | loc             |   batch_size |   learning_rate |   num_epochs |
|---------------------+----------+-----------------+--------------+-----------------+--------------|
| DEFAULT_d5cba_00000 | RUNNING  | 127.0.0.1:14084 |           50 |     0.000109278 |           10 |
| DEFAULT_d5cba_00001 | PENDING  |                 |           50 |     0.0422751   |            5 |
| DEFAULT_d5cba_00002 | PENDING  |                 |     

  0%|          | 0/9 [00:00<?, ?it/s]


...

Result for DEFAULT_d5cba_00009:
  accuracy: 0.6833333333333333
  date: 2021-12-22_15-45-22
  done: false
  experiment_id: be12fc2152074c17bd0b7a8da6051ed8
  hostname: WSTOMAS21
  iterations_since_restore: 4
  loss: 0.8536348938941956
  node_ip: 127.0.0.1
  pid: 31648
  should_checkpoint: true
  time_since_restore: 44.42823028564453
  time_this_iter_s: 9.967573881149292
  time_total_s: 44.42823028564453
  timestamp: 1640184322
  timesteps_since_restore: 0
  training_iteration: 4
  trial_id: d5cba_00009
  


 57%|█████▋    | 24/42 [00:04<00:01, 10.35it/s]


== Status ==
Current time: 2021-12-22 15:45:27 (running for 00:06:55.03)
Memory usage on this node: 17.7/47.6 GiB
Using AsyncHyperBand: num_stopped=7
Bracket: Iter 8.000: -0.8849095702171326 | Iter 4.000: -0.8792380392551422 | Iter 2.000: -0.9153856784105301 | Iter 1.000: -0.962437543604109
Resources requested: 1.0/24 CPUs, 1.0/2 GPUs, 0.0/37.98 GiB heap, 0.0/18.99 GiB objects
Result logdir: C:\Users\Tomas\ray_results\DEFAULT_2021-12-22_15-38-31
Number of trials: 10/10 (1 RUNNING, 9 TERMINATED)
+---------------------+------------+-----------------+--------------+-----------------+--------------+----------+
| Trial name          | status     | loc             |   batch_size |   learning_rate |   num_epochs |     loss |
|---------------------+------------+-----------------+--------------+-----------------+--------------+----------|
| DEFAULT_d5cba_00009 | RUNNING    | 127.0.0.1:31648 |           10 |     0.0454214   |            5 | 0.853635 |
| DEFAULT_d5cba_00000 | TERMINATED | 127.0.0

100%|██████████| 18/18 [00:04<00:00,  3.68it/s]
2021-12-22 15:45:32,114	INFO tune.py:626 -- Total run time: 420.67 seconds (419.92 seconds for the tuning loop).


Result for DEFAULT_d5cba_00009:
  accuracy: 0.6833333333333333
  date: 2021-12-22_15-45-31
  done: false
  experiment_id: be12fc2152074c17bd0b7a8da6051ed8
  hostname: WSTOMAS21
  iterations_since_restore: 5
  loss: 0.8538116961717606
  node_ip: 127.0.0.1
  pid: 31648
  should_checkpoint: true
  time_since_restore: 54.341593742370605
  time_this_iter_s: 9.913363456726074
  time_total_s: 54.341593742370605
  timestamp: 1640184331
  timesteps_since_restore: 0
  training_iteration: 5
  trial_id: d5cba_00009
  
Result for DEFAULT_d5cba_00009:
  accuracy: 0.6833333333333333
  date: 2021-12-22_15-45-31
  done: true
  experiment_id: be12fc2152074c17bd0b7a8da6051ed8
  experiment_tag: 9_batch_size=10,learning_rate=0.045421,num_epochs=5
  hostname: WSTOMAS21
  iterations_since_restore: 5
  loss: 0.8538116961717606
  node_ip: 127.0.0.1
  pid: 31648
  should_checkpoint: true
  time_since_restore: 54.341593742370605
  time_this_iter_s: 9.913363456726074
  time_total_s: 54.341593742370605
  timestamp